In [1]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import csv


In [2]:
df = pd.read_csv('IAS_dataset.csv', sep=';')
print(df.head())

     Id        date  year    Home Team  Away Team Score  \
0  1042  2016-08-13  2016        STOKE    CHELSEA   1-2   
1  1209  2016-08-13  2017    NEWCASTLE  LIVERPOOL   1-1   
2  1744  2016-08-13  2018  SOUTHAMPTON    EVERTON   2-1   
3  1780  2016-08-13  2018      EVERTON   MAN CITY   0-2   
4  1928  2016-08-13  2019       WOLVES    BURNLEY   1-1   

   Home Team Goals Scored  Away Team Goals Scored  Home Team Goals Conceeded  \
0                       1                       2                          2   
1                       1                       1                          1   
2                       2                       1                          1   
3                       0                       2                          2   
4                       1                       1                          1   

   Away Team Goals Conceeded  ... Away Team Fouls  Away Team Yellow Cards  \
0                          1  ...            11.0                     2.0   
1         

In [3]:
with open('IAS_dataset.csv', 'r') as file:
    reader = csv.reader(file)
    attributes = next(reader)  # lit la première ligne du fichier (les noms d'attributs)
    features = attributes[0].split(";")[1:]  # extrait les noms d'attributs de la première ligne
    print("Liste des attributs :", features)




Liste des attributs : ['date', 'year', 'Home Team', 'Away Team', 'Score', 'Home Team Goals Scored', 'Away Team Goals Scored', 'Home Team Goals Conceeded', 'Away Team Goals Conceeded', 'Half Time Score', 'Match Excitement', 'Home Team Rating', 'Away Team Rating', 'Home Team Possession %', 'Away Team Possession %', 'Home Team Off Target Shots', 'Home Team On Target Shots', 'Home Team Total Shots', 'Home Team Blocked Shots', 'Home Team Corners', 'Home Team Throw Ins', 'Home Team Pass Success %', 'Home Team Aerials Won', 'Home Team Clearances', 'Home Team Fouls', 'Home Team Yellow Cards', 'Home Team Second Yellow Cards', 'Home Team Red Cards', 'Away Team Off Target Shots', 'Away Team On Target Shots', 'Away Team Total Shots', 'Away Team Blocked Shots', 'Away Team Corners', 'Away Team Throw Ins', 'Away Team Pass Success %', 'Away Team Aerials Won', 'Away Team Clearances', 'Away Team Fouls', 'Away Team Yellow Cards', 'Away Team Second Yellow Cards', 'Away Team Red Cards', 'xG Home Team', 'xG

In [4]:
hTeams = df['Home Team'].unique() 

nbGoalsHome = df["Home Team Goals Scored"]

AvgNbGoalsHome = np.mean(nbGoalsHome)

df.describe()


,Id,year,Home Team Goals Scored,Away Team Goals Scored,Home Team Goals Conceeded,Away Team Goals Conceeded,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,...,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,xG Home Team,xG Away Team,NonShot xG Home Team,NonShot xG Away Team,SPI Rate Home Team,SPI Rate Away Team
count,1900.000000,1900.000000,1900.000000,1900.0000,1900.0000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,...,1900.000000,1900.000000,1900.000000,1900.000000,1899.000000,1899.000000,1899.000000,1899.000000,1900.000000,1900.000000
mean,1709.500000,2018.000000,1.513158,1.2300,1.2300,1.513158,5.355579,6.421105,6.216158,51.192105,...,10.796316,1.708421,0.027368,0.036842,1.554676,1.275176,1.541348,1.273054,70.213442,70.139032
std,548.627074,1.414586,1.307545,1.2096,1.2096,1.307545,1.675894,0.996768,0.992519,13.056100,...,3.532753,1.264925,0.163197,0.199289,0.904789,0.794524,0.805929,0.676804,11.907131,11.912981
min,760.000000,2016.000000,0.000000,0.0000,0.0000,0.000000,1.700000,2.400000,1.500000,18.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.030000,42.970000,43.410000
25%,1234.750000,2017.000000,1.000000,0.0000,0.0000,1.000000,4.100000,5.700000,5.500000,42.000000,...,8.000000,1.000000,0.000000,0.000000,0.850000,0.680000,0.970000,0.780000,60.847500,60.562500
50%,1709.500000,2018.000000,1.000000,1.0000,1.0000,1.000000,5.300000,6.400000,6.200000,51.000000,...,11.000000,2.000000,0.000000,0.000000,1.410000,1.130000,1.420000,1.170000,68.955000,68.795000
75%,2184.250000,2019.000000,2.000000,2.0000,2.0000,2.000000,6.400000,7.000000,6.800000,61.000000,...,13.000000,3.000000,0.000000,0.000000,2.120000,1.745000,1.965000,1.640000,80.275000,80.180000
max,2659.000000,2020.000000,9.000000,9.0000,9.0000,9.000000,10.000000,10.000000,10.000000,83.000000,...,26.000000,7.000000,1.000000,2.000000,7.070000,5.100000,6.580000,4.310000,95.750000,95.470000


In [5]:
label = df.columns[-1]
X = df[features].values
Y = df[label].values

In [6]:
df.head()

,Id,date,year,Home Team,Away Team,Score,Home Team Goals Scored,Away Team Goals Scored,Home Team Goals Conceeded,Away Team Goals Conceeded,...,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,xG Home Team,xG Away Team,NonShot xG Home Team,NonShot xG Away Team,SPI Rate Home Team,SPI Rate Away Team
0,1042,2016-08-13,2016,STOKE,CHELSEA,1-2,1,2,2,1,...,11.0,2.0,0.0,0.0,1.27,2.30,0.78,2.16,50.70,82.23
1,1209,2016-08-13,2017,NEWCASTLE,LIVERPOOL,1-1,1,1,1,1,...,6.0,1.0,0.0,0.0,0.37,2.25,0.33,1.82,55.73,77.55
2,1744,2016-08-13,2018,SOUTHAMPTON,EVERTON,2-1,2,1,1,2,...,11.0,1.0,0.0,0.0,1.22,0.87,0.97,1.40,64.77,66.86
3,1780,2016-08-13,2018,EVERTON,MAN CITY,0-2,0,2,2,0,...,6.0,1.0,0.0,0.0,2.13,2.11,0.71,2.11,58.69,79.47
4,1928,2016-08-13,2019,WOLVES,BURNLEY,1-1,1,1,1,1,...,11.0,2.0,0.0,0.0,1.51,1.49,0.99,1.32,74.08,68.99


In [7]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [8]:
from sklearn.preprocessing import OrdinalEncoder

# création d'un encodeur ordinal
encoder = OrdinalEncoder()

# ajustement de l'encodeur sur les noms des équipes
encoder.fit(df[["Home Team", "Away Team"]])

# transformation des noms des équipes en nombres
df[["Home Team", "Away Team"]] = encoder.transform(df[["Home Team", "Away Team"]])
df[["Home Team", "Away Team"]].head()


,Home Team,Away Team
0,22.0,6.0
1,18.0,14.0
2,21.0,8.0
3,8.0,15.0
4,29.0,4.0


In [9]:
df = df.dropna() 
df["Match Outcome"] = df["Home Team Goals Scored"] - df["Away Team Goals Scored"]
df["Match Outcome"] = df["Match Outcome"].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
#df = pd.get_dummies(df, columns=["Home Team", "Away Team"]).head() # Convertir les attributs catégoriels en attributs numériques

# Sélection des attributs d'entraînement
df = df.drop("date", axis = 1)
df = df.drop("Score", axis = 1)
df = df.drop("Half Time Score", axis = 1)



In [10]:
df.head()

,Id,year,Home Team,Away Team,Home Team Goals Scored,Away Team Goals Scored,Home Team Goals Conceeded,Away Team Goals Conceeded,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,xG Home Team,xG Away Team,NonShot xG Home Team,NonShot xG Away Team,SPI Rate Home Team,SPI Rate Away Team,Match Outcome
0,1042,2016,22.0,6.0,1,2,2,1,6.7,5.1,7.3,36,64,2.0,1.0,5.0,2.0,5.0,22.0,67.0,13.0,17.0,16.0,5.0,1.0,0.0,5.0,7.0,20.0,8.0,11.0,23.0,84.0,16.0,29.0,11.0,2.0,0.0,0.0,1.27,2.30,0.78,2.16,50.70,82.23,-1
1,1209,2017,18.0,14.0,1,1,1,1,4.8,6.6,6.3,32,68,1.0,5.0,8.0,2.0,1.0,21.0,66.0,17.0,35.0,11.0,1.0,0.0,0.0,9.0,2.0,17.0,6.0,5.0,24.0,84.0,20.0,14.0,6.0,1.0,0.0,0.0,0.37,2.25,0.33,1.82,55.73,77.55,0
2,1744,2018,21.0,8.0,2,1,1,2,5.7,7.2,5.8,39,61,4.0,4.0,11.0,3.0,7.0,22.0,68.0,40.0,41.0,12.0,2.0,0.0,0.0,1.0,2.0,7.0,4.0,6.0,36.0,74.0,31.0,14.0,11.0,1.0,0.0,0.0,1.22,0.87,0.97,1.40,64.77,66.86,1
3,1780,2018,8.0,15.0,0,2,2,0,4.6,5.7,7.5,39,61,0.0,1.0,4.0,3.0,5.0,26.0,75.0,17.0,23.0,12.0,1.0,0.0,0.0,4.0,4.0,15.0,7.0,6.0,23.0,86.0,29.0,25.0,6.0,1.0,0.0,0.0,2.13,2.11,0.71,2.11,58.69,79.47,-1
4,1928,2019,29.0,4.0,1,1,1,1,5.1,6.0,6.1,65,35,8.0,2.0,17.0,7.0,4.0,21.0,79.0,39.0,23.0,9.0,0.0,0.0,0.0,6.0,4.0,13.0,3.0,3.0,12.0,66.0,23.0,25.0,11.0,2.0,0.0,0.0,1.51,1.49,0.99,1.32,74.08,68.99,0


In [11]:
df =df.drop("Home Team Goals Scored", axis = 1)
df = df.drop("Away Team Goals Scored", axis = 1)
df = df.drop("Home Team Goals Conceeded", axis = 1)
df = df.drop("Away Team Goals Conceeded", axis = 1)
df.head()

,Id,year,Home Team,Away Team,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,xG Home Team,xG Away Team,NonShot xG Home Team,NonShot xG Away Team,SPI Rate Home Team,SPI Rate Away Team,Match Outcome
0,1042,2016,22.0,6.0,6.7,5.1,7.3,36,64,2.0,1.0,5.0,2.0,5.0,22.0,67.0,13.0,17.0,16.0,5.0,1.0,0.0,5.0,7.0,20.0,8.0,11.0,23.0,84.0,16.0,29.0,11.0,2.0,0.0,0.0,1.27,2.30,0.78,2.16,50.70,82.23,-1
1,1209,2017,18.0,14.0,4.8,6.6,6.3,32,68,1.0,5.0,8.0,2.0,1.0,21.0,66.0,17.0,35.0,11.0,1.0,0.0,0.0,9.0,2.0,17.0,6.0,5.0,24.0,84.0,20.0,14.0,6.0,1.0,0.0,0.0,0.37,2.25,0.33,1.82,55.73,77.55,0
2,1744,2018,21.0,8.0,5.7,7.2,5.8,39,61,4.0,4.0,11.0,3.0,7.0,22.0,68.0,40.0,41.0,12.0,2.0,0.0,0.0,1.0,2.0,7.0,4.0,6.0,36.0,74.0,31.0,14.0,11.0,1.0,0.0,0.0,1.22,0.87,0.97,1.40,64.77,66.86,1
3,1780,2018,8.0,15.0,4.6,5.7,7.5,39,61,0.0,1.0,4.0,3.0,5.0,26.0,75.0,17.0,23.0,12.0,1.0,0.0,0.0,4.0,4.0,15.0,7.0,6.0,23.0,86.0,29.0,25.0,6.0,1.0,0.0,0.0,2.13,2.11,0.71,2.11,58.69,79.47,-1
4,1928,2019,29.0,4.0,5.1,6.0,6.1,65,35,8.0,2.0,17.0,7.0,4.0,21.0,79.0,39.0,23.0,9.0,0.0,0.0,0.0,6.0,4.0,13.0,3.0,3.0,12.0,66.0,23.0,25.0,11.0,2.0,0.0,0.0,1.51,1.49,0.99,1.32,74.08,68.99,0


In [12]:
features = ['Id', 'year', 'Home Team', 'Away Team', 'Match Excitement',
       'Home Team Rating', 'Away Team Rating', 'Home Team Possession %',
       'Away Team Possession %', 'Home Team Off Target Shots',
       'Home Team On Target Shots', 'Home Team Total Shots',
       'Home Team Blocked Shots', 'Home Team Corners', 'Home Team Throw Ins',
       'Home Team Pass Success %', 'Home Team Aerials Won',
       'Home Team Clearances', 'Home Team Fouls', 'Home Team Yellow Cards',
       'Home Team Second Yellow Cards', 'Home Team Red Cards',
       'Away Team Off Target Shots', 'Away Team On Target Shots',
       'Away Team Total Shots', 'Away Team Blocked Shots', 'Away Team Corners',
       'Away Team Throw Ins', 'Away Team Pass Success %',
       'Away Team Aerials Won', 'Away Team Clearances', 'Away Team Fouls',
       'Away Team Yellow Cards', 'Away Team Second Yellow Cards',
       'Away Team Red Cards', 'xG Home Team', 'xG Away Team',
       'NonShot xG Home Team', 'NonShot xG Away Team', 'SPI Rate Home Team',
       'SPI Rate Away Team']

In [13]:


# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df[features], df["Match Outcome"], test_size=0.1, random_state=42)

# Entraînement d'un modèle de forêt aléatoire
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Calcul de la précision
accuracy = accuracy_score(y_test, y_pred)
print("Précision : {:.2f}%".format(accuracy * 100))



Précision : 91.58%


In [14]:
X_test.head()

,Id,year,Home Team,Away Team,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,xG Home Team,xG Away Team,NonShot xG Home Team,NonShot xG Away Team,SPI Rate Home Team,SPI Rate Away Team
1222,1200,2017,10.0,25.0,5.9,4.9,8.4,40,60,2.0,1.0,6.0,3.0,4.0,31.0,73.0,11.0,25.0,7.0,1.0,0.0,0.0,5.0,7.0,14.0,2.0,7.0,30.0,81.0,14.0,31.0,4.0,1.0,0.0,0.0,0.24,1.34,0.19,2.16,47.99,81.97
111,1195,2017,4.0,10.0,1.9,6.1,6.7,52,48,1.0,2.0,4.0,1.0,3.0,31.0,73.0,26.0,22.0,9.0,3.0,0.0,0.0,5.0,2.0,9.0,2.0,4.0,31.0,76.0,22.0,35.0,14.0,2.0,0.0,0.0,0.50,0.27,0.67,0.49,60.30,47.25
554,2075,2019,1.0,21.0,6.5,5.1,7.1,56,44,7.0,6.0,19.0,6.0,7.0,22.0,78.0,31.0,22.0,16.0,2.0,0.0,0.0,5.0,8.0,17.0,4.0,5.0,21.0,70.0,19.0,23.0,15.0,1.0,0.0,0.0,1.27,2.60,2.19,1.10,65.24,65.80
526,2507,2020,14.0,15.0,6.0,5.0,7.8,56,44,2.0,3.0,8.0,3.0,6.0,25.0,88.0,10.0,9.0,13.0,2.0,0.0,0.0,3.0,5.0,8.0,0.0,1.0,11.0,85.0,6.0,23.0,8.0,1.0,0.0,0.0,1.01,1.13,1.25,0.75,69.90,73.12
1311,988,2016,22.0,8.0,5.0,6.3,6.0,47,53,5.0,7.0,14.0,2.0,5.0,19.0,78.0,23.0,23.0,21.0,1.0,0.0,0.0,7.0,5.0,15.0,3.0,5.0,20.0,79.0,14.0,26.0,12.0,0.0,0.0,0.0,1.68,0.98,1.69,0.86,63.01,71.06


In [15]:
correct_predictions = []
incorrect_predictions = []

for i in range(len(y_pred)):
    if y_pred[i] == y_test.values[i]:
        correct_predictions.append(X_test.index[i])
    else:
        incorrect_predictions.append(X_test.index[i])

print("Nombre de prédictions correctes :", len(correct_predictions))
print("Nombre de prédictions incorrectes :", len(incorrect_predictions))

# Affichage des prédictions correctes
""" print("Prédictions correctes :")
for i in correct_predictions:
    print("Match", i, "prédiction :", y_pred[X_test.index.get_loc(i)], "résultat réel :", y_test[i])

# Affichage des prédictions incorrectes
print("Prédictions incorrectes :")
for i in incorrect_predictions:
    print("Match", i, "prédiction :", y_pred[X_test.index.get_loc(i)], "résultat réel :", y_test[i]) """


Nombre de prédictions correctes : 174
Nombre de prédictions incorrectes : 16


' print("Prédictions correctes :")\nfor i in correct_predictions:\n    print("Match", i, "prédiction :", y_pred[X_test.index.get_loc(i)], "résultat réel :", y_test[i])\n\n# Affichage des prédictions incorrectes\nprint("Prédictions incorrectes :")\nfor i in incorrect_predictions:\n    print("Match", i, "prédiction :", y_pred[X_test.index.get_loc(i)], "résultat réel :", y_test[i]) '

In [16]:
X_test.head()

,Id,year,Home Team,Away Team,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,xG Home Team,xG Away Team,NonShot xG Home Team,NonShot xG Away Team,SPI Rate Home Team,SPI Rate Away Team
1222,1200,2017,10.0,25.0,5.9,4.9,8.4,40,60,2.0,1.0,6.0,3.0,4.0,31.0,73.0,11.0,25.0,7.0,1.0,0.0,0.0,5.0,7.0,14.0,2.0,7.0,30.0,81.0,14.0,31.0,4.0,1.0,0.0,0.0,0.24,1.34,0.19,2.16,47.99,81.97
111,1195,2017,4.0,10.0,1.9,6.1,6.7,52,48,1.0,2.0,4.0,1.0,3.0,31.0,73.0,26.0,22.0,9.0,3.0,0.0,0.0,5.0,2.0,9.0,2.0,4.0,31.0,76.0,22.0,35.0,14.0,2.0,0.0,0.0,0.50,0.27,0.67,0.49,60.30,47.25
554,2075,2019,1.0,21.0,6.5,5.1,7.1,56,44,7.0,6.0,19.0,6.0,7.0,22.0,78.0,31.0,22.0,16.0,2.0,0.0,0.0,5.0,8.0,17.0,4.0,5.0,21.0,70.0,19.0,23.0,15.0,1.0,0.0,0.0,1.27,2.60,2.19,1.10,65.24,65.80
526,2507,2020,14.0,15.0,6.0,5.0,7.8,56,44,2.0,3.0,8.0,3.0,6.0,25.0,88.0,10.0,9.0,13.0,2.0,0.0,0.0,3.0,5.0,8.0,0.0,1.0,11.0,85.0,6.0,23.0,8.0,1.0,0.0,0.0,1.01,1.13,1.25,0.75,69.90,73.12
1311,988,2016,22.0,8.0,5.0,6.3,6.0,47,53,5.0,7.0,14.0,2.0,5.0,19.0,78.0,23.0,23.0,21.0,1.0,0.0,0.0,7.0,5.0,15.0,3.0,5.0,20.0,79.0,14.0,26.0,12.0,0.0,0.0,0.0,1.68,0.98,1.69,0.86,63.01,71.06


In [17]:
# Créer un DataFrame avec les colonnes "Home Team" et "Away Team" de X_test
teams_encoded = X_test[["Home Team", "Away Team"]]

# Utiliser l'encodeur pour inverser la transformation sur les colonnes "Home Team" et "Away Team"
teams_decoded = encoder.inverse_transform(teams_encoded)

# Remplacer les colonnes "Home Team" et "Away Team" de X_test avec les noms d'équipe décodés
X_test[["Home Team", "Away Team"]] = teams_decoded

# Afficher X_test pour vérifier que les noms d'équipe ont été décodés avec succès
print(X_test[["Home Team", "Away Team"]].head())


         Home Team     Away Team
1222  HUDDERSFIELD     TOTTENHAM
111        BURNLEY  HUDDERSFIELD
554    ASTON VILLA   SOUTHAMPTON
526      LIVERPOOL      MAN CITY
1311         STOKE       EVERTON


In [18]:
""" print("Prédictions correctes :")
for i in correct_predictions:
    print("Match", i, "prédiction :", y_pred[X_test.index.get_loc(i)], "résultat réel :", y_test[i]) """

# Affichage des prédictions incorrectes
print("Prédictions incorrectes :")
for i in incorrect_predictions:
    print("Match :", X_test["Home Team"].values[X_test.index.get_loc(i)],"-", X_test["Away Team"].values[X_test.index.get_loc(i)] ,"prédiction :", y_pred[X_test.index.get_loc(i)], "résultat réel :", y_test[i])

Prédictions incorrectes :
Match : STOKE - EVERTON prédiction : 1 résultat réel : 0
Match : EVERTON - SOUTHAMPTON prédiction : 1 résultat réel : 0
Match : CHELSEA - ARSENAL prédiction : 1 résultat réel : 0
Match : LEICESTER CITY - NORWICH prédiction : 1 résultat réel : 0
Match : STOKE - LEICESTER CITY prédiction : 1 résultat réel : 0
Match : BURNLEY - LEICESTER CITY prédiction : 1 résultat réel : 0
Match : WATFORD - BURNLEY prédiction : 1 résultat réel : 0
Match : TOTTENHAM - MAN UTD prédiction : -1 résultat réel : 0
Match : MAN UTD - ARSENAL prédiction : 0 résultat réel : 1
Match : WOLVES - TOTTENHAM prédiction : 1 résultat réel : 0
Match : WEST BROM - MAN UTD prédiction : 0 résultat réel : -1
Match : BURNLEY - HULL CITY prédiction : -1 résultat réel : 0
Match : SHEFFIELD UTD - CRYSTAL PALACE prédiction : 0 résultat réel : 1
Match : LEICESTER CITY - WEST HAM prédiction : 1 résultat réel : 0
Match : BRIGHTON - CRYSTAL PALACE prédiction : 0 résultat réel : -1
Match : CRYSTAL PALACE - MID

In [19]:
X_test["Home Team"].values[1]

'BURNLEY'

In [20]:
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
model.score(X_train,y_train)


1.0